In [1]:
import pandas as pd

df = pd.read_csv('subset_tweets_preprocessed.csv')
df.head(-5)

,tweet_id,text,account,domain,background,label
0,1603396689552965635,fútbolplaya copaconosur 🇦🇷🇧🇴🇺🇾🇨🇱 argentina enf...,@Argentina,PERSON,SPORT,ARG
1,1608210531713695745,opinión en cuestiones de igualdad de género sa...,@publico_es,ORG,NEW,ES
2,1608881936512032768,picaenpunta el club inglés quiere llevarse al ...,@cronica,ORG,NEW,ARG
3,1550218833729933312,mira este video sus segundos importan y merece...,@AlejandroSanz,PERSON,ART,ES
4,1608743487448522752,la transparencia de ratajkowski e increíble ha...,@laSextaTV,ORG,NEW,ES
...,...,...,...,...,...,...
9974,1002124654897778688,el sr sánchez habla de la mala situación de es...,@marianorajoy,PERSON,POL,ES
9975,1608846311989682176,con la confirmación de la joven de 20 años en ...,@publico_es,ORG,NEW,ES
9976,1564381204601679873,así actuaron el sábado con el liderazgo de mar...,@horaciorlarreta,PERSON,POL,ARG
9977,1499405957125881858,…te ame te amo y te amare luz de todos los ast...,@FitoPaezMusica,PERSON,ART,ARG


In [2]:
group_counts = df['background'].value_counts()
print("Groups in the 'background' column:")
print(group_counts)


Groups in the 'background' column:
POL      4089
NEW      3544
ART       941
SPORT     741
MEDIA     669
Name: background, dtype: int64


In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    DistilBertConfig,
    DistilBertForSequenceClassification,
    Trainer,
    TrainingArguments,
)
from datasets import Dataset
from datasets.load import load_dataset, load_metric
import os


# Opening Dataset
df = pd.read_csv('subset_tweets_preprocessed.csv')

# Load the dataset from the Hugging Face datasets library
dataset = load_dataset('csv', data_files='tweets_preprocessed.csv', delimiter=',')

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained("dccuchile/distilbert-base-spanish-uncased")

label_mapping = {'POL': 0, 'NEW': 1, 'ART': 2, 'MEDIA': 3, 'SPORT': 4}

# Encoding Data
def process_data(row):
    text = row['text']
    text = str(text)
    text = ' '.join(text.split())
    encodings = tokenizer(text, padding="max_length", truncation=True, max_length=128)
    label = row['background']
    label_encoding = label_mapping[label]  # Map the label to its numerical value
    encodings['labels'] = label_encoding
    encodings['text'] = text
    return encodings

processed_data = [process_data(row) for _, row in df.iterrows()]

# Shuffle the dataset
import random
random.shuffle(processed_data)

# Splitting Dataset
train, test = train_test_split(processed_data, test_size=0.2)


train_dict = {k: [example[k] for example in train] for k in train[0]}
test_dict = {k: [example[k] for example in test] for k in test[0].keys()}

# Create separate output directory for predictions
output_dir = "./predictions"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
train_dataset = Dataset.from_dict(train_dict)
test_dataset = Dataset.from_dict(test_dict)

# Config and Model
config = DistilBertConfig.from_pretrained(
    "dccuchile/distilbert-base-spanish-uncased", num_labels=5, output_hidden_states=True
)
model = DistilBertForSequenceClassification.from_pretrained(
    "dccuchile/distilbert-base-spanish-uncased", config=config,
)

# Data Collator
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, padding=True)

# Metric and Metric computation function
metric = load_metric("f1")

def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions
    if isinstance(preds, tuple):
        preds = preds[0]
    preds = np.argmax(preds, axis=-1)
    f1 = metric.compute(predictions=preds, references=labels, average="macro")
    return f1


# Training Arguments
training_args = TrainingArguments(
    output_dir="./multiclass_stuff",
    evaluation_strategy="steps",
    eval_steps=10,
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=32,
    num_train_epochs=3,
    weight_decay=0.01,
    push_to_hub=False,
    logging_dir="./logs",
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="f1",
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

# Evaluation
eval_results = trainer.predict(test_dataset)
eval_metric = compute_metrics(eval_results)
print(eval_metric)

Found cached dataset csv (C:/Users/LENOVO/.cache/huggingface/datasets/csv/default-c400035869c40f6e/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/1 [00:00<?, ?it/s]

Some weights of the model checkpoint at dccuchile/distilbert-base-spanish-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at dccuchile/distilbert-base-spanish-uncased and are newly initialized: ['pre_classifier.wei

Step,Training Loss,Validation Loss,F1
10,1.602100,1.504214,0.112830
20,1.392200,1.344427,0.195914
30,1.263200,1.233206,0.267267
40,1.237700,1.177497,0.276343
50,1.182200,1.171401,0.222058
60,1.135900,1.101475,0.274608
70,1.071000,1.085688,0.266776
80,1.009600,1.040292,0.281610
90,1.159400,0.990463,0.299200
100,0.960500,0.963207,0.297587


{'f1': 0.7240149225787047}
